In [9]:
import pandas as pd
import numpy as np

In [24]:
ratings_data = pd.read_csv('ml-1m/ratings.dat',sep='::',names=['userID','movieID','rating','timestamp'])

/home/parijat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [25]:
movie_data = pd.read_csv('ml-1m/movies.dat',sep='::',names=['movieId','title','genres'])

/home/parijat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [26]:
users_data = pd.read_csv('ml-1m/users.dat',sep='::',names=['userID','gender','age','occupation','zip-code'])

/home/parijat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [27]:
users_data.head()

,userID,gender,age,occupation,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [28]:
ratings_data.head()

,userID,movieID,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [22]:
movie_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [139]:
print(users_data.shape,movie_data.shape,ratings_data.shape)

(6040, 5) (3883, 5) (1000209, 4)


In [140]:
users_data['userID'].unique()

array([   1,    2,    3, ..., 6038, 6039, 6040])

In [141]:
movie_data['movieId'].nunique()

3883

In [142]:
users_data['gender'] = users_data['gender'].apply(lambda x: 1 if x=='M' else 0)

In [143]:
movie_data['year'] = movie_data['title'].apply(lambda x:x.split(' ')[-1])

In [144]:
movie_data['year'] = movie_data['year'].apply(lambda x:np.int(x[-5:-1]))
#movie_data['dem'] = movie_data['year'].apply(lambda x:True if len(x)>4 else False)

In [146]:
movie_data['genres'] = movie_data['genres'].apply(lambda x: x.split('|'))

In [147]:
movie_data

,movieId,title,genres,year,dem
0,1,Toy Story (1995),"[Animation, Children's, Comedy]",1995,False
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]",1995,False
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995,False
3,4,Waiting to Exhale (1995),"[Comedy, Drama]",1995,False
4,5,Father of the Bride Part II (1995),[Comedy],1995,False
5,6,Heat (1995),"[Action, Crime, Thriller]",1995,False
6,7,Sabrina (1995),"[Comedy, Romance]",1995,False
7,8,Tom and Huck (1995),"[Adventure, Children's]",1995,False
8,9,Sudden Death (1995),[Action],1995,False
9,10,GoldenEye (1995),"[Action, Adventure, Thriller]",1995,False


In [154]:
ratings_data['month']=ratings_data['timestamp'].apply(lambda x:pd.Timestamp(x).month)

In [156]:
ratings_data['month'].value_counts()

1    1000209
Name: month, dtype: int64

In [164]:
ratings_data['second']=ratings_data['timestamp'].apply(lambda x:pd.Timestamp(x).second)

In [165]:
ratings_data['second'].value_counts()

0    962251
1     37958
Name: second, dtype: int64

In [169]:
all_genres = {}
for i,row in movie_data.iterrows():
    
    for genre in row['genres']:
        if genre not in all_genres:
            all_genres[genre] = 1
            
movie_data = pd.concat([movie_data,pd.DataFrame(data=np.zeros((movie_data.shape[0],len(all_genres.keys()))),columns=all_genres.keys())],axis=1)

In [171]:
movie_data.shape

(3883, 23)

In [184]:
for i,row in movie_data.iterrows():
    for genre in row['genres']:
        movie_data.loc[i,genre]=1.0
        

In [185]:
movie_data.head()

,movieId,title,genres,year,dem,Animation,Children's,Comedy,Adventure,Fantasy,...,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1,Toy Story (1995),"[Animation, Children's, Comedy]",1995,False,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]",1995,False,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995,False,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),"[Comedy, Drama]",1995,False,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),[Comedy],1995,False,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
